In [2]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

In [3]:
includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### General Parameters

In [4]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
all_res, plants, parts = read_data(filepath_Ljungan)
print() 

### River System  
We will work with Sydkraft in this example  
For ease at first we have one producer, one reservoir and one plant.

In [5]:
j = parts[1]
res = filter(r -> j.participationrate[r] > 0, all_res)[1]
k = j.plants[1]

println("Participant: ", j)
println("Reservoir: ", res)
println("Power Plant: ", k)

Participant: Sydkraft

Reservoir: Flasjon

Power Plant: Flasjo



### Bidding Model  
At the Electricity Markets, Participants have to communicate increasing bidding curves of price-volume pairs before market clearing.  
Depending on the market clearing price, the delivery of each participant becomes known  
We approximate this relation by using linear interpolation of volumes and presetting price Points, for example based on probabilities.

$$
y_t = \frac{c_t - P_{t,i} }{P_{t,i+1} - P_{t,i}} \cdot x_{i,t} + \frac{P_{t,i+1} - c_t }{P_{t,i+1} - P{t,i}} \cdot x_{i+1,t}, \qquad \text{if} \qquad P_{t,i} \leq c_t \leq P_{t,i+1}
$$

The Volumes have to be in increasing order:

$$
x_{i,t} \leq x_{i+1,t}
$$

### Hydro Model

The Commodity we deal with is electricity. At all times it has to be in balance and consumed at the moment it is generated. The means of producing electricity is a pure hydro system with a reservoir upstream for storage.  

$$
\begin{align*}
l &= \text{Reservoir level} \\
Q &= \text{Flow of Water} \\
w &= \text{Electrical Production} \\
e &= \text{Equivalent}
\end{align*}
$$

### Set Parameters necessary for Input into Model

In [6]:
Stages = 8 # 2 lowest number as first stage is just to achieve nonanticipativity
T = 24
PPoints = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
I = length(PPoints)-1

println("Stages :" , Stages, "\n Hours: ", T, "\n I : ", I, "\n Price Points = ", [i for i in PPoints])

Qref = Dict{Reservoir, Float64}(r => 10.0 for r in [res])
scenario_count = 3
Prices = [floor.(rand(T), sigdigits=3) for i in 1:scenario_count]
Inflows = [10.0]
Omega = [(price = p, inflow = v) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Costs
S = 0.1
# Cost for Up and Down Balancing
mu_up = 0.7
mu_down = 0.3

println("Uncertainty Set: ", Omega)

Stages :8
 Hours: 24
 I : 5


 Price Points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]


Uncertainty Set: 

NamedTuple{(:price, :inflow), Tuple{Vector{Float64}, Float64}}[(price = [0.155, 0.945, 0.83, 0.203, 0.524, 0.74, 0.0125, 0.306, 0.814, 0.0905, 0.309, 0.304, 0.411, 0.69, 0.43, 0.152, 0.602, 0.29, 0.708, 0.817, 0.987, 0.392, 0.327, 0.182], inflow = 10.0), (price = [0.567, 0.198, 0.891, 0.0824, 0.204, 0.799, 0.285, 0.401, 0.711, 0.649, 0.0786

, 0.276, 0.692, 0.882, 0.247, 0.114, 0.728, 0.0811, 0.202, 0.267, 0.764, 0.56, 0.147, 0.00738], inflow = 10.0), (price = [0.861, 0.885, 0.44, 0.645, 0.666, 0.257, 0.682, 0.557, 0.228, 0.531, 0.211, 0.399, 0.841, 0.558, 0.577, 0.663, 0.577, 0.867, 0.597, 0.223, 0.931, 0.876, 0.289, 0.996], inflow

 = 10.0)]


### Definition of Subproblems

In [7]:
function subproblem_builder(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = res.currentvolume)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, w[t=1:T] >= 0)
    @variable(subproblem, b[t=1:T] >= 0)
    @variable(subproblem, Q[t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.in == l.out)
    else
        @constraint(subproblem, obligation[t=1:T], y[t] == w[t] + b[t])
        @constraint(subproblem, balance, l.out == l.in - sum(Q[t] for t in 1:T) + f * T - s)
        @constraint(subproblem, production[t=1:T], w[t] == Q[t] * k.equivalent)
        SDDP.parameterize(subproblem, Omega, P) do om
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * b[t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder (generic function with 1 method)

### Build the SDDP Model

In [8]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

A policy graph with 8 nodes.
 Node indices: 1, 2, 3, 4, 5, 6, 7, 8


### Train the model

In [9]:
SDDP.train(model; iteration_limit = 100)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------


problem
  nodes           : 8
  state variables : 145
  scenarios       : 2.18700e+03
  existing cuts   : false
options
  solver          : 

serial mode
  risk measure    : 

SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [389, 389]
  AffExpr in MOI.EqualTo{Float64}         : [25, 73]
  AffExpr in MOI.LessThan{Float64}        : [120, 120]
  VariableRef in MOI.GreaterThan{Float64} : [243, 244]
  VariableRef in MOI.LessThan{Float64}    : [146, 146]


numerical stability report
  matrix range     [5e-03, 2e+01]
  objective range  [7e-03, 1e+00]
  bounds range     [5e+01, 1e+05]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


...

  1
 

┌ Warning: Found a cut with a mix of small and large coefficients.
│     The order of magnitude difference is 13.289999502043637.
│     The smallest cofficient is -2.6201263381153694e-14.
│     The largest coefficient is 0.5108833333333334.
│ 
│ You can ignore this warning, but it may be an indication of numerical issues.
│ 
│ Consider rescaling your model by using different units, e.g, kilometers instead
│ of meters. You should also consider reducing the accuracy of your input data (if
│ you haven't already). For example, it probably doesn't make sense to measure the
│ inflow into a reservoir to 10 decimal places.
└ @ SDDP C:\Users\Lenni\.julia\packages\SDDP\ppkik\src\plugins\bellman_functions.jl:68


       35   2.782623e+03  2.878752e+03  1.442100e+01      2650   1


        40   2.631093e+03  2.878752e+03  1.469300e+01      2800   1
-------------------------------------------------------------------
status         : 

simulation_stopping
total time (s) : 1.469300e+01
total solves   : 2800
best bound     :  2.878752e+03
simulation ci  :  2.719838e+03 ± 1.944760e+02
numeric issues : 0
-------------------------------------------------------------------



### Obtain the Bidding and Nomination Decisions

In [10]:
rule = SDDP.DecisionRule(model; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:l => res.currentvolume),
    #incoming_state = merge(Dict(Symbol("l_real[$(r)]") => r.currentvolume for r in res), Dict(Symbol("l_ind[$(r)]") => r.currentvolume for r in res)),
    controls_to_record = [:Q, :y, :x, :l]
)
println(solution.controls[:x])
print(solution.controls[:l])

SDDP.State{Float64}[

SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}

(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP

.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0

, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}

...

, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64

}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{

Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{

Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)]


SDDP.State{Float64}(35000.0, 35000.0)

In [11]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:x, :y, :w, :l, :b, :Q, :f],
)

100-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:b => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :l => SDDP.State{Float64}(35000.0, 35000.0), :w => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :bellman_term => 2878.7522717333322, :node_index => 1, :objective_state => nothing, :belief => Dict(1 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); … ; SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)], :stage_objective => 0, :Q => 

In [12]:
replication = 28
stage = 8
println("Bidding Curve Values: ", simulations[replication][stage][:x])
println("Cleared Volume: ", simulations[replication][stage][:y])
println("Reservoir Level: ", simulations[replication][stage][:l])
println("Own Production: ", simulations[replication][stage][:w])
println("Water Used: ", simulations[replication][stage][:Q])
println("Purchases: ", simulations[replication][stage][:b])
println("Filling in that stage: ", simulations[replication][stage][:f])

Bidding Curve Values: SDDP.State{Float64}[SDDP.State{Float64}(50.0

, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.

State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP

.State{Float64}(50.0, 0.0); SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.

State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0); SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0

...

(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.

State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0)]


Cleared Volume: [38.74999999999999, 50.0, 50.0, 0.7500000000000007, 31.000000000000007, 34.99999999999999, 3.125, 26.499999999999996, 50.0, 22.624999999999996, 27.249999999999996, 25.999999999999996, 2.7499999999999893, 22.499999999999982, 7.499999999999994, 37.99999999999999, 0.5000000000000002, 22.499999999999993, 26.999999999999986, 50.0, 50.0, 48.0, 31.75, 45.49999999999999]
Reservoir Level: SDDP.State{Float64}(2259.9999999999986, 219.35483870967636)


Own Production: [38.74999999999999, 50.0, 50.0, 0.7500000000000007, 31.000000000000007, 34.99999999999999, 3.125, 26.499999999999996, 50.0, 22.624999999999996, 27.249999999999996, 25.999999999999996, 2.7499999999999893, 22.499999999999982, 7.499999999999994, 37.99999999999999, 0.5000000000000002, 22.499999999999993, 26.999999999999986, 50.0, 50.0, 48.0, 31.75, 45.49999999999999]
Water Used: [124.99999999999999, 161.29032258064518, 161.29032258064518, 2.4193548387096797, 100.00000000000003, 112.9032258064516, 10.080645161290324, 85.48387096774194, 161.29032258064518, 72.98387096774194, 87.9032258064516, 83.87096774193547, 8.87096774193545, 72.58064516129028, 24.193548387096754, 122.5806451612903, 1.6129032258064524, 72.5806451612903, 87.09677419354834, 161.29032258064518, 161.29032258064518, 154.83870967741936, 102.41935483870968, 146.77419354838707]
Purchases: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

 0.0, 0.0, 0.0]
Filling in that stage: 10.0


### Inclusion of Water Regulation Rules

The Water Regulation Rules have to be accounted for when bidding at the Market

* We communicate a sequence of bids at the electricity markets for every hour of the day
* The Water Regulation company receives ONE Bid for discharge over the entire day

For currrent purposes, we assume that the nomination is fixed and can't be changed, and the amount of water has to actually be used over the course of the day.  

* We have to buy electricity on the market for certain hours if our nomination was too low
* We have to sell surplus of water if our nomination was too high. The price on the balancing market is by assumption very low

For that we include an additional State Variable: $Q^\text{nom}_r$ for every reservoir. Let $Q_{r,t}$ the real flow of water from $r$ at hour $t$.   
For every reservoir it has to hold that:  

$$
T \cdot Q^\text{nom}_r = \sum\limits_{t=1}^T Q_{r,t}
$$

We change the objective Function to the following:


$$
\max \; \rightarrow \; \sum\limits_{t = 1}^T y_{i_t, t} \cdot c_t - (\mu^+ \cdot z^+_{i_t,t}  - \mu^- \cdot z^-_{i_t,t})
$$
The coefficient $\mu^+, \mu^-$ are chosen so that they are higher than the highest or lower than the lowest observed spot market price respectively. 
Accordingly, the delivery obligation constraint is changed to (assuming there is one river and one power plant): 

$$
y_{i_t, t} = Q_{r,t} \cdot e_k + z^+_{t} - z^-_{t}  
$$

The difficulty in the agents decision is that the nomination $Q^\text{nom}_r$ has to be done before market clearing, thus it is modelled as a state variable.

In [13]:
function subproblem_builder_regulation(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = 1000)
    @variable(subproblem, 0 <= Qnom, SDDP.State, initial_value = 0)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, w[t=1:T] >= 0)
    @variable(subproblem, z_up[t=1:T] >= 0)
    @variable(subproblem, z_down[t=1:T] >= 0)
    @variable(subproblem, Q[t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.out == l.in - Qnom.out - s) 
    else
        @constraint(subproblem, nomination, sum(Q[t] for t in 1:T) == T * Qnom.in)
        @constraint(subproblem, obligation[t=1:T], y[t] == w[t] + z_up[t] - z_down[t])
        @constraint(subproblem, balance, l.out == l.in - T * Qnom.out + f * T - s)
        @constraint(subproblem, production[t=1:T], w[t] == Q[t] * k.equivalent)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * z_up[t] + mu_down * z_down[t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder_regulation (generic function with 1 method)

In [14]:
model3 = SDDP.LinearPolicyGraph(
    subproblem_builder_regulation;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

SDDP.train(model3; iteration_limit=100)
rule = SDDP.DecisionRule(model3; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:l => res.currentvolume),
    controls_to_record = [:x, :Qnom]
)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 146
  scenarios       : 2.18700e+03
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [415, 415]
  AffExpr in MOI.EqualTo{Float64}         : [25, 74]
  AffExpr in MOI.LessThan{Float64}        : [120, 120]
  VariableRef in MOI.GreaterThan{Float64} : [268, 269]
  VariableRef in MOI.LessThan{Float64}    : [146, 146]


numerical stability report
  matrix range     [5e-03, 2e+01]
  objective range  [7e-03, 1e+00]
  bounds range     [5e+01, 1e+05]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   4.257500e+02  6.149001e+03  4.600000e-02        30   1


         6   3.048689e+03  3.047463e+03  1.076000e+00       980   1


        21   3.011056e+03  3.047463e+03  2.495000e+00      2230   1


        40   3.008240e+03  3.047463e+03  3.338000e+00      2800   1
-------------------------------------------------------------------
status         : simulation_stopping
total time (s) : 3.338000e+00
total solves   : 2800
best bound     :  3.047463e+03
simulation ci  :  2.997143e+03 ± 1.319684e+02
numeric issues : 0
-------------------------------------------------------------------



(stage_objective = 0, outgoing_state = Dict(Symbol("x[2,24]") => 0.0, Symbol("x[3,7]") => 50.0, Symbol("x[5,19]") => 50.0, Symbol("x[5,3]") => 50.0, Symbol("x[3,5]") => 50.0, Symbol("x[5,7]") => 50.0, Symbol("x[3,8]") => 50.0, Symbol("x[3,2]") => 0.0, Symbol("x[4,14]") => 50.0, Symbol("x[6,10]") => 50.0…), controls = Dict{Symbol, Any}(:Qnom => SDDP.State{Float64}(0.0, 35000.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0); … ; SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)]))

In [15]:
SDDP.write_subproblem_to_file(model3[2], "subproblem_2.mof.json")
sbproblem = JuMP.read_from_file("subproblem_2.mof.json")
sbproblem

A JuMP Model
Maximization problem with:
Variables: 415
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 74 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 122 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 147 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 267 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 146 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [16]:
simulations = SDDP.simulate(
    model3,
    100,
    [:x, :y, :z_up, :z_down, :w, :Qnom, :Q, :l]
)

100-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:l => SDDP.State{Float64}(1000.0, 0.0), :w => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :bellman_term => 3047.463499999999, :node_index => 1, :objective_state => nothing, :z_up => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :belief => Dict(1 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0); … ; SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)], :stage_objective => 0, :z_down => [0.0,

In [17]:
replication = 1
stage = 2

simulations[replication][stage][:Qnom]

SDDP.State{Float64}(1000.0, 0.0)

### Increased Complexity of Hydro System  

The system of creating electricity using hydropower can be modelled using higher complexity than a constant power coefficient. So far this does not reflect:

* Multiple Reservoirs
* Multiple Power Plants at one reservoir
* Generators at Power Plants beeing on/off
* Start-Up Costs of Reservoirs

For that we have to:  

* Adapt the production constraint: Production is now not only relation of water, but also if generators are turned on
$$
\begin{align*}
 W^\text{min} \cdot u_{k,t} &\leq w_{k,t} \leq  W^\text{max} \cdot u_{k,t} = e_k \cdot Q^\text{spill}_k \cdot \\
w_{k,t} &= Q^\text{eff}_{r,t} \cdot e_k \\
Q^\text{nom}_{r} &\leq \max\limits_{k \in \mathcal{K}} Q^\text{spill} \\
Q^\text{eff}_{k,t} & \leq \sum\limits_{r' \in \mathcal{R}^\text{us}_k} Q^\text{real}_{r,t}
 \end{align*}
$$

The bound on the nomination is there, so that the nominations don't get unnecessarily large. This is already discouraged by the objective, but valuable tight bound on variables are still relevant.

In [18]:
function subproblem_builder_hydro(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = res.currentvolume)
    @variable(subproblem, 0 <= Qnom <= k.spillreference, SDDP.State, initial_value = 0)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, u[t=1:T], Bin)
    @variable(subproblem, 0 <= w[t=1:T] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t=1:T] >= 0)
    @variable(subproblem, z_down[t=1:T] >= 0)
    @variable(subproblem, Qreal[t=1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t=1:T] <= k.spillreference)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.out == l.in - Qnom.out - s) 
    else
        @constraint(subproblem, nomination, sum(Qreal[t] for t in 1:T) == T * Qnom.in)
        @constraint(subproblem, obligation[t=1:T], y[t] == w[t] + z_up[t] - z_down[t])
        @constraint(subproblem, balance, l.out == l.in - T * Qnom.out + f * T - s)
        @constraint(subproblem, active[t=1:T], w[t] == u[t] * k.spillreference * k.equivalent)
        @constraint(subproblem, production[t=1:T], w[t] == Qeff[t] * k.equivalent)
        @constraint(subproblem, realwater[t=1:T], Qeff[t] <= Qreal[t] )
        @constraint(subproblem, spillwater[t=1:T], Qeff[t] <= k.spillreference)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * z_up[t] + mu_down * z_down[t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder_hydro (generic function with 1 method)

In [19]:
model4 = SDDP.LinearPolicyGraph(
    subproblem_builder_hydro;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

SDDP.train(model4; iteration_limit=10)
rule = SDDP.DecisionRule(model4; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:l => res.maxvolume),
    controls_to_record = [:x, :Qnom]
)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 146
  scenarios       : 2.18700e+03
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [463, 463]
  AffExpr in MOI.EqualTo{Float64}         : [25, 98]
  AffExpr in MOI.LessThan{Float64}        : [120, 168]
  VariableRef in MOI.GreaterThan{Float64} : [292, 293]
  VariableRef in MOI.LessThan{Float64}    : [195, 195]
  VariableRef in MOI.ZeroOne              : [24, 24]


numerical stability report
  matrix range     [5e-03, 2e+01]
  objective range  [7e-03, 1e+00]
  bounds range     [2e-01, 1e+05]
  rhs range        [6e-01, 6e-01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   2.837500e+02  7.226314e+02  1.207000e+00        30   1


         2   1.577049e+01  4.670285e+02  5.336000e+00       860   1


        10   3.345902e+02  3.687304e+02  6.129000e+00      1100   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 6.129000e+00
total solves   : 1100
best bound     :  3.687304e+02
simulation ci  :  2.441509e+02 ± 5.806555e+01
numeric issues : 0
-------------------------------------------------------------------



(stage_objective = 0, outgoing_state = Dict(Symbol("x[2,24]") => 0.0, Symbol("x[3,7]") => 50.0, Symbol("x[5,19]") => 50.0, Symbol("x[5,3]") => 50.0, Symbol("x[3,5]") => 0.0, Symbol("x[5,7]") => 50.0, Symbol("x[3,8]") => 26.293102171499818, Symbol("x[3,2]") => 0.0, Symbol("x[4,14]") => 0.0, Symbol("x[6,10]") => 50.0…), controls = Dict{Symbol, Any}(:Qnom => SDDP.State{Float64}(0.0, 0.58), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0); … ; SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)]))

### Adding mulitple reservoirs and multiple Power Plants

Final step is to 
* parametrize the reservoirs and the power plants.  
The flow of water across one river now goes through multiple power plants, and water can come from reservoirs much further upstream.
* Inclusion of Start-Up Costs  
We include the additional restriction
$$
\delta_{k,h} \geq u_{k,h+1} - u_{k,h}, \qquad \forall k \in \mathcal{K}, \; \forall h = 1,\ldots,T-1
$$
And penalize start-up costs by an additional parameter $S_k$ in the objective. We additionally deal with a unit commitment problem of which power plants to use.
* First and last status of hydropower plant as state variable  
Otherwise there would be no start-up cost in the first hour of the next stage, if the power plant wasn't running at night. Depending on the significance of the start-up cost, that makes a difference.  
* Inclusion of Individual Reservoir and max nomination for negative reservoirs  
We have to generalize the indices of our constraints for this new problem, by indexing reservoirs $r \in \mathcal{R}$ and power plants $k \in \mathcal{K}$.

In [20]:
# These are new Parameters for the problem
T = 24
Stages = 8
R = [res]
K = j.plants
S = 0.2
Qref = Dict(r => 0.5 for r in R)
println("Reservoirs: ", R)
println("Power Plants: ", K)
println(filter(k -> k.reservoir == res, K))
println("The minimum spillreference at the reservoirs is simply: ", min([k.spillreference for k in filter(k -> k.reservoir == res, K)]...))

Reservoirs: 

Reservoir[Flasjon
]
Power Plants: 

HydropowerPlant[Flasjo
, Trangfors
, Ratan
, Turinge
, Bursnas
]


HydropowerPlant[Flasjo
, Trangfors
, Ratan
, Turinge
, Bursnas
]
The minimum spillreference at the reservoirs is simply: 0.58


In [27]:
function subproblem_builder_hydro_multiple(subproblem::Model, node::Int64)
    set_attribute(subproblem, "CPX_PARAM_EPINT", 1e-2)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l[r = R] <= r.maxvolume, SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, lind[r = R], SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, u_start[k = K], SDDP.State, initial_value = 0, Bin)
    @variable(subproblem, 0 <= Qnom[r = R] <= min([k.spillreference for k in filter(k -> k.reservoir == res, K)]...), SDDP.State, initial_value = 0)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, d[t=1:T, k = K], Bin)
    @variable(subproblem, u[t=1:T, k = K], Bin)
    @variable(subproblem, BALANCE_INDICATOR[r = R], Bin)
    @variable(subproblem, 0 <= w[t=1:T, k = K] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t=1:T] >= 0)
    @variable(subproblem, z_down[t=1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t=1:T, k = K] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t=1:T, r = R])
    @variable(subproblem, f[r = R] >= 0)
    @variable(subproblem, s[r = R] >= 0)

    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    @constraint(subproblem, balance_ind[r = R], lind[r].out == lind[r].in - T * (Qnom[r].out - Qref[r])- s[r]) 
    @constraint(subproblem, nbal1[r = R], BALANCE_INDICATOR[r] => {Qnom[r].out <= Qref[r]})
    @constraint(subproblem, nbal2[r = R], !BALANCE_INDICATOR[r] => {0 <= lind[r].in})
    @constraint(subproblem, NoSpill[k = K], BALANCE_INDICATOR[k.reservoir] => {sum(Qnom[r_up].out for r_up in find_us_reservoir(k.reservoir)) <= k.spillreference})
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer[r = R], l[r].out == l[r].in - T * Qnom[r].out - s[r]) 
    else
        @constraint(subproblem, endcond[k = K], u_start[k].out == u[T,k])
        @constraint(subproblem, startcond[k = K], u_start[k].in == u[1,k])
        @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
        @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qnom[r].in)
        @constraint(subproblem, obligation[t=1:T], y[t] == sum(w[t,k] for k in K) + z_up[t] - z_down[t])
        @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qnom[r].out + f[r] * T - s[r])
        @constraint(subproblem, active[t=1:T, k=K], w[t,k] == u[t,k] * k.spillreference * k.equivalent)
        @constraint(subproblem, startup[t=1:T-1, k=K], d[t,k] >= u[t+1,k] - u[t,k])
        @constraint(subproblem, production[t=1:T, k=K], w[t,k] == Qeff[t,k] * k.equivalent)
        @constraint(subproblem, realwater[t=1:T, k=K], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
        @constraint(subproblem, spillwater[t=1:T, k=K], Qeff[t,k] <= k.spillreference)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            for r in R
                JuMP.fix(f[r], om.inflow, force=true)
            end
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * z_up[t] + mu_down * z_down[t]  - S * sum(d[t,k] for k in K) for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder_hydro_multiple (generic function with 1 method)

In [22]:
println(Omega)

NamedTuple{(:price, :inflow), Tuple{Vector{Float64}, Float64}}[(price = [0.155, 0.945, 0.83, 0.203, 0.524, 0.74, 0.0125, 0.306, 0.814, 0.0905, 0.309, 0.304, 0.411, 0.69, 0.43, 0.152, 0.602, 0.29, 0.708, 0.817, 0.987, 0.392, 0.327, 0.182], inflow = 10.0), (price = [0.567, 0.198, 0.891, 0.0824, 0.204, 0.799, 0.285, 0.401, 0.711, 0.649, 0.0786, 0.276, 0.692, 0.882, 0.247, 0.114, 0.728, 0.0811, 0.202, 0.267,

 0.764, 0.56, 0.147, 0.00738], inflow = 10.0), (price = [0.861, 0.885, 0.44, 0.645, 0.666, 0.257, 0.682, 0.557, 0.228, 0.531, 0.211, 0.399, 0.841, 0.558, 0.577, 0.663, 0.577, 0.867, 0.597, 0.223, 0.931, 0.876, 0.289, 0.996], inflow = 10.0)]


In [33]:
model5 = SDDP.LinearPolicyGraph(
    subproblem_builder_hydro_multiple;
    stages = 3,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

SDDP.train(model5; iteration_limit=10, duality_handler = SDDP.ContinuousConicDuality())
# rule = SDDP.DecisionRule(model5; node = 1)
# solution = SDDP.evaluate(
#     rule;
#     incoming_state = Dict(Symbol("l[$(r)]") => r.maxvolume for r in R),
#     controls_to_record = [:z_up]
# )

# SDDP.parameterize(model5[2],(price = Prices[3], inflow= 10.0))
# SDDP.write_subproblem_to_file(model5[2], "subproblem_2.mof.json")
# sbproblem = JuMP.read_from_file("subproblem_2.mof.json")
# println(sbproblem)


-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 3


  state variables : 152
  scenarios       : 9.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [884, 884]
  AffExpr in MOI.EqualTo{Float64}                                               : [2, 301]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 115]
  AffExpr in MOI.LessThan{Float64}                                              : [120, 360]
  VariableRef in MOI.GreaterThan{Float64}                                       : [484, 485]
  VariableRef in MOI.LessThan{Float64}                                          : [387, 387]
  VariableRef in MOI.ZeroOne                                                    : [246, 246]
  Vector{AffExpr} in MOI.Indicator{MOI.ACTIVATE_ON_ONE, MOI.LessThan{Float64}}  : [6, 6]
  Vector{AffExpr} in MOI.Indicator{

numerical stability report
  matrix range     [5e-03, 2e+01]
  objective range  [7e-03, 1e+00]
  bounds range     [7e-02, 1e+05]
  rhs range        [6e-01, 1e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   3.730000e+01  2.496705e+02  9.899998e-02        10   1


         2   6.649526e+01  1.571023e+02  5.838000e+00        47   1


         4   8.154445e+01  1.302432e+02  7.021000e+00        67   1


         6   7.720157e+01  1.277448e+02  8.330000e+00        87   1


         8   6.783310e+01  1.258702e+02  9.450000e+00       107   1


        10   7.336793e+01  1.251475e+02  1.081700e+01       127   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 1.081700e+01
total solves   : 127
best bound     :  1.251475e+02
simulation ci  :  7.959531e+01 ± 1.773623e+01
numeric issues : 0
-------------------------------------------------------------------



In [24]:
SDDP.simulate(model5, 100)[1][2]

Dict{Symbol, Any} with 6 entries:
  :bellman_term    => 61.0858
  :noise_term      => (price = [0.155, 0.945, 0.83, 0.203, 0.524, 0.74, 0.0125,…
  :node_index      => 2
  :stage_objective => 52.1025
  :objective_state => nothing
  :belief          => Dict(2=>1.0)